# Performing training and deployment of a KNN model using the CSV and RecordIO Protobuf training input formats

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r X_train
%store -r y_train
%store -r X_test
%store -r y_test

In [ ]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()
session_bucket = session.default_bucket()

In [ ]:
from sagemaker import KNN

estimator1 = KNN(
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    sample_size=50,
    k=3,
    feature_dim=5,
    predictor_type="classifier",
    sagemaker_session=session)

In [ ]:
record_set = estimator1.record_set(train=X_train, labels=y_train)

In [ ]:
estimator1.fit(record_set)

In [ ]:
predictor1 = estimator1.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

In [ ]:
X_test

In [ ]:
results1 = predictor1.predict(X_test)
results1[0:3]

In [ ]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter04/knn"

In [ ]:
!aws s3 cp tmp/train.io s3://{s3_bucket}/{prefix}/input/train.io

In [ ]:
from sagemaker.inputs import TrainingInput

train_path = f"s3://{s3_bucket}/{prefix}/input/train.io"
train = TrainingInput(train_path, 
                      content_type="application/x-recordio-protobuf")

In [ ]:
from sagemaker.estimator import Estimator

estimator2 = Estimator(
    image_uri=estimator1.training_image_uri(),
    role=role,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    sagemaker_session=session)

In [ ]:
estimator2.set_hyperparameters(
    sample_size=50,
    k=3,
    predictor_type="classifier")

In [ ]:
estimator2.fit({"train": train})

In [ ]:
predictor2 = estimator2.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

In [ ]:
import sagemaker
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor2.serializer = CSVSerializer()
predictor2.deserializer = JSONDeserializer()

In [ ]:
results2 = predictor2.predict(X_test)
results2["predictions"][0:3]

In [ ]:
predictor1.delete_endpoint()

In [ ]:
predictor2.delete_endpoint()